In [1]:
import torch
import gpytorch
import time
import math
import datetime
import pickle
import sys
parent_folder = 'D:/BOMI-master/BOMissingInputs' # Path to the project's root
sys.path.append(parent_folder)

import pandas as pd
from BOMI.BOGPyTorch.GPTmodels import ExactGPModel
from scipy.stats import mode

from BOMI.MyBPMF.MatrixFactorization import myBPMF
from BOMI.ndfunction import *

from sklearn.neighbors import KernelDensity

# from missingpy import KNNImputer

mySeed = 9
np.random.seed(mySeed)
torch.random.manual_seed(mySeed)


class BOTorch():
    def __init__(self):
        return

    @staticmethod
    def optimizeHyperparamaters(in_model, train_x, train_y, optimizerAlgo, trainingIter):
        # Set train data
        train_x 
        train_y 
        in_model 
        in_model.set_train_data(train_x, train_y)
        # Find optimal model hyperparameters
        in_model.train()
        in_model.likelihood.train()
        # Use the adam optimizer
        if optimizerAlgo == "Adam":
            optimizer = torch.optim.Adam([{'params': in_model.parameters()}, ],
                                         lr=0.1)  # Includes GaussianLikelihood parameters
        elif optimizerAlgo == "SGD":
            optimizer = torch.optim.SGD([{'params': in_model.parameters()}, ],
                                        lr=0.1)  # Includes GaussianLikelihood parameters
        elif optimizerAlgo == "LBFGS":
            optimizer = torch.optim.LBFGS([{'params': in_model.parameters()}, ], lr=0.1, history_size=50, max_iter=10,
                                          line_search_fn=True)
        # "Loss" for GPs - the marginal log likelihood
        mll = gpytorch.mlls.ExactMarginalLogLikelihood(in_model.likelihood, in_model)
        training_iter = trainingIter

        # LBFGS
        # define closure
        if optimizerAlgo == "LBFGS":
            def closure():
                optimizer.zero_grad()
                output = in_model(train_x)
                loss = -mll(output, train_y)
                return loss

            loss = closure()
            loss.backward()

            for i in range(training_iter):
                # perform step and update curvature
                loss = optimizer.step(closure)
                loss.backward()

        elif optimizerAlgo == "SGD" or optimizerAlgo == "Adam":
            # SGD & Adam
            for i in range(training_iter):
                # Zero gradients from previous iteration
                optimizer.zero_grad()
                # Output from model
                output = in_model(train_x)
                # Calc loss and backprop gradients
                loss = -mll(output, train_y)
                loss.backward()
                optimizer.step()


    @staticmethod
    def posterior_predict(in_x, in_model):
        in_model.eval()
        in_model.likelihood.eval()
        # Calculate posterior predictions
        f_preds = in_model(in_x)
        # Take mean and variance
        tmpMean = f_preds.mean
        tmpVar = f_preds.variance
        # Get from tensors
        mean = tmpMean.detach().numpy()
        var = tmpVar.detach().numpy()
        return mean, var

    @staticmethod
    def posterior_predict_scalar(in_x, in_model):
        in_model.eval()
        in_model.likelihood.eval()
        mean = []
        var = []
        for point in in_x:
            # Push to tensor
            point = torch.tensor([point.item()]) 
            # Calculate posterior predictions
            f_preds = in_model(point)
            # Take mean and variance
            tmpMean = f_preds.mean
            tmpVar = f_preds.variance
            # Get from tensors
            mean.append(tmpMean.item())
            var.append(tmpVar.item())
        return mean, var

    @staticmethod
    def gpucb(in_x, in_model, in_Beta):
        in_model.eval()
        in_model.likelihood.eval()
        x = torch.tensor(in_x, dtype=torch.float32) 
        # Calculate posterior predictions
        f_preds = in_model(x)
        # Take mean and variance
        tmpMean = f_preds.mean
        tmpVar = f_preds.variance
        # Calculate acquisition
        tmpAcq = tmpMean + torch.tensor(in_Beta)  * tmpVar
        # Get from tensors
        acq = tmpAcq.clone().detach().cpu().numpy()
        tmpMean = tmpMean.clone().detach().cpu().numpy()
        tmpVar = tmpVar.clone().detach().cpu().numpy()
        return acq, tmpMean, tmpVar

    @staticmethod
    def gpucb_scalar(in_x, in_model, in_Beta):
        in_model.eval()
        in_model.likelihood.eval()
        for i in range(len(in_x)):
            in_x[i] = float(in_x[i])
        x = torch.tensor([in_x], dtype=torch.float32)
        # print("type x:", x)
        # Calculate posterior predictions
        f_preds = in_model(x)
        # Take mean and variance
        tmpMean = f_preds.mean
        tmpVar = f_preds.variance
        # Calculate acquisition
        tmpAcq = tmpMean + torch.tensor(in_Beta) * tmpVar
        # Get from tensors
        acq = tmpAcq.detach().cpu().numpy()
        return acq

# Input arguments
input_method = "BOMI" # method = "Drop", "Suggest", "BPMF", "BOMI", "Mean", "Mode", "KNN", "uGP"
input_function = "Schubert4d" # Black-box objective functions, see file 'ndfunction.py'
input_numGPs = int(5) # Default: 5 (GPs)
input_in_alpha = float(1e2) # Default: 1e2 - BPMF parameter
input_missingRate = float(0.25) # 0.25
input_missingNoise = float(0.05) # 0.05

# Select true objective function
missRate = input_missingRate
missNoise = input_missingNoise
if input_function == "Eggholder2d":
    myfunc = Eggholder(2, missRate, missNoise, False)
elif input_function == "Schubert4d":
    myfunc = Schubert_Nd(4, missRate,missNoise, False)
elif input_function == "Alpine5d":
    myfunc = Alpine_Nd(5,missRate,missNoise,False)
elif input_function == "Schwefel5d":
    myfunc = Schwefel_Nd(5, missRate,missNoise, False)

# GP-UCB parameters
scaleBeta = 0.2
delt = 0.1
a = 1.0
b = 1.0
dim = myfunc.input_dim
r = 1.0

BetaMu = 1.0
BetaSigma = 1.0

# Experiments settings
Runs = 10
if input_function == "Eggholder2d":
    NumIterations = 61
elif input_function == "Schubert4d":
    NumIterations = 121
elif input_function == "Alpine5d" or input_function == "Schwefel5d":
    NumIterations = 161


log_iter = ""
log_iter_point = ""
log_run = ""

# Select method
method = input_method
isMulMethod = True
numGPs = input_numGPs
GPHypers_optim = "Adam"
GPoptimIter = 50
totalTimeStart = time.time()
for run in range(0, Runs):
    # Read data from file
    if input_function == "Eggholder2d":
        dfX = pd.read_csv(r'data/Eggholder2d/Eggholder2dX_' + str(run) + '.csv')
    elif input_function == "Schubert4d":
        dfX = pd.read_csv(r'data/Schubert4d/SchubertNd4dX_' + str(run) + '.csv')
    elif input_function == "Alpine5d":
        dfX = pd.read_csv(r'data/Alpine5d/AlpineNd5dX_' + str(run) + '.csv')
    elif input_function == "Schwefel5d":
        dfX = pd.read_csv(r'data/Schwefel5d/SchwefelNd5dX_' + str(run) + '.csv')

    inX = dfX.values
    Xori = inX.tolist().copy()

    if input_function == "Eggholder2d":
        dfY = pd.read_csv(r'data/Eggholder2d/EggholderY.csv')
    elif input_function == "Schubert4d":
        dfY = pd.read_csv(r'data/Schubert4d/SchubertNdY.csv')
    elif input_function == "Alpine5d":
        dfY = pd.read_csv(r'data/Alpine5d/AlpineNdY.csv')
    elif input_function == "Schwefel5d":
        dfY = pd.read_csv(r'data/Schwefel5d/SchwefelNdY.csv')

    inY = dfY.values
    inY = inY.squeeze()
    Yori = inY.tolist().copy()

    na = 0.0
    nb = 1.0
    minY = np.min(Yori)
    maxY = np.max(Yori)
    normalizedY = [[(((nb - na) * (tmpY - minY) / (maxY - minY) + na))] for tmpY in Yori]

    normalizedX = [myfunc.normalize(xi) for xi in Xori]

    R = np.append(normalizedX, normalizedY, axis=1)
    R[np.isnan(R)] = -1

    # Arrays of observations after dropping/removing
    dropNX = []
    dropNY = []
    dropYori = []
    dropXori = []
    for i in range(len(normalizedX)):
        if -1 not in R[i]:
            dropNX.append(normalizedX[i])
            dropNY.append(normalizedY[i])
            dropXori.append(Xori[i])
            dropYori.append(Yori[i])

    if method == "Drop" or method == "Suggest" or method == "uGP":
        in_X = dropXori
        in_Y = dropYori
        minY = np.min(in_Y)
        maxY = np.max(in_Y)
        if minY == maxY:
            minY -= 0.01
        n_in_Y = [(((nb - na) * (tmpY - minY) / (maxY - minY) + na)) for tmpY in in_Y]
        n_in_X = [myfunc.normalize(xi) for xi in in_X]
    else:
        in_X = Xori
        in_Y = Yori
        n_in_X = normalizedX
        n_in_Y = normalizedY

    # Initialize likelihood and model
    train_x = torch.tensor(n_in_X, dtype=torch.float32)
    train_y = torch.tensor(n_in_Y, dtype=torch.float32)
    myGP = ExactGPModel(train_x, train_y) 

    # Init BO objects:
    myBO = BOTorch()

    if method == "BPMF" or method == "BOMI":
        R_in = np.append(n_in_X, n_in_Y, axis=1)
        R_in[np.isnan(R_in)] = -1
        D = 15
        (N, M) = R.shape
        T = 40

        beta0 = None
        myBPMFObject = myBPMF()

    in_Guessed_X = dropXori
    in_Guessed_Y = dropYori
    in_Guessed_nY = [(((nb - na) * (tmpY - np.min(in_Guessed_Y)) / (np.max(in_Guessed_Y) - np.min(in_Guessed_Y)) + na))
                     for tmpY in in_Guessed_Y]
    in_Guessed_nX = [myfunc.normalize(xi) for xi in in_Guessed_X]

    ####################### BO loops #######################
    myfunc.numCalls = 0
    BOstart_time = time.time()
    for ite in range(NumIterations):
        precalBetaT = 2.0 * np.log((ite + 1) * (ite + 1) * math.pi ** 2 / (3 * delt)) + 2 * dim * np.log(
            (ite + 1) * (ite + 1) * dim * b * r * np.sqrt(np.log(4 * dim * a / delt)))
        BetaT = np.sqrt(precalBetaT) * scaleBeta

        # Train the GP model
        # BPMF require filling missing values first before feeding into the GP
        if method == "BPMF":
            R_in = np.append(n_in_X, n_in_Y, axis=1)
            R_in[np.isnan(R_in)] = -1
            newX = []
            newY = []

            (N, M) = R_in.shape
            U_in = np.zeros((D, N))
            V_in = np.zeros((D, M))

            in_alpha = input_in_alpha * r
            R_pred, train_err_list, Rs = myBPMFObject.ProposedBPMF(R_in, R_in, U_in, V_in, T, D,
                                                                   initial_cutoff=0, lowest_rating=0.0,
                                                                   highest_rating=1.0, in_alpha=in_alpha,
                                                                   numSamples=numGPs,
                                                                   Beta_0=beta0, output_file=None,
                                                                   missing_mask=-1,
                                                                   save_file=False)
            # Use the new predicted matrix as input training data for the GP
            res = R_pred.copy()
            newX = np.delete(res, myfunc.input_dim, axis=1)
            newY = (np.delete(res, 0, axis=1)).tolist()
            for _ in range(myfunc.input_dim - 2):
                newY = (np.delete(newY, 0, axis=1)).tolist()
            newY = ((np.delete(newY, 0, axis=1)).reshape(1, -1)).tolist()

            train_x = torch.tensor(newX, dtype=torch.float32) 
            train_y = torch.tensor(newY, dtype=torch.float32) 

            myGP = ExactGPModel(train_x, train_y) 
            myBO.optimizeHyperparamaters(myGP, train_x, train_y, GPHypers_optim, GPoptimIter)
        elif method == "BOMI":
            BPMFstart_time = time.time()

            R_in = np.append(n_in_X, n_in_Y, axis=1)
            R_in[np.isnan(R_in)] = -1

            newX = []
            newY = []

            (N, M) = R_in.shape
            U_in = np.zeros((D, N))
            V_in = np.zeros((D, M))

            in_alpha = input_in_alpha * r
            R_pred, train_err_list, Rs = myBPMFObject.ProposedBPMF(R_in, R_in, U_in, V_in, T, D,
                                                                   initial_cutoff=0, lowest_rating=0.0,
                                                                   highest_rating=1.0, in_alpha=in_alpha,
                                                                   numSamples=numGPs,
                                                                   Beta_0=beta0, output_file=None,
                                                                   missing_mask=-1,
                                                                   save_file=False)

            BPMFstop_time = time.time()

            PRstart_time = time.time()
            idxs = np.where(R_in == -1)
            for ii in range(numGPs):
                tmpR = R_in.copy()
                for iii in range(len(idxs[0])):
                    tmpR[idxs[0][iii], idxs[1][iii]] = Rs[ii][idxs[0][iii]][idxs[1][iii]]
                Rs[ii] = tmpR

            PRstop_time = time.time()

            # Use the new predicted matrix as input training data for the GP
            BuildGPstart_time = time.time()
            GPs = []
            for ii in range(numGPs):
                res = Rs[ii].copy()
                newX = np.delete(res, myfunc.input_dim, axis=1)
                newY = (np.delete(res, 0, axis=1)).tolist()
                for _ in range(myfunc.input_dim - 2):
                    newY = (np.delete(newY, 0, axis=1)).tolist()
                newY = ((np.delete(newY, 0, axis=1)).reshape(1, -1)).tolist()

                train_x = torch.tensor(newX, dtype=torch.float32) 
                train_y = torch.tensor(newY, dtype=torch.float32) 
                tmpGP = ExactGPModel(train_x, train_y) 

                myBO.optimizeHyperparamaters(tmpGP, train_x, train_y, GPHypers_optim, GPoptimIter)
                GPs.append(tmpGP)

            BuildGPstop_time = time.time()
            # Duong add vao

            opt_sug_starttime = time.time()
            # Sample the next point from each GP
            testX = [myfunc.randUniformInNBounds() for i in range(10000)]

            candidatesX = testX.copy()
            testX = torch.tensor(testX)
            nextXs = []
            nextXsAcq = []

            acqs = []
            for ii in range(numGPs):
                GPs[ii].eval()
                GPs[ii].likelihood.eval()
                acq, mean_i, var_i = myBO.gpucb(testX, GPs[ii], BetaT)
                acqs.append(acq)

            sAcq = np.mean(acqs, axis=0) + BetaT * np.std(acqs, axis=0)

            tmpX = testX[np.argmax(sAcq)]
            tmpX = tmpX.detach().cpu().numpy()
            nextXs.append(tmpX.tolist())
            nextXsAcq.append(sAcq[np.argmax(sAcq)])

            nextX = nextXs[np.argmax(nextXsAcq)]

            opt_sug_stoptime = time.time()
            print("nextX:", nextX, " idx:", np.argmax(nextXsAcq))


        elif method == "Mean":

            R_in = np.append(n_in_X, n_in_Y, axis=1)
            R_in[np.isnan(R_in)] = -1
            idxs = np.where(R_in == -1)

            tmpR = R_in.copy()
            meansR = np.mean(tmpR, axis=0)
            for iii in range(len(idxs[0])):
                tmpR[idxs[0][iii], idxs[1][iii]] = meansR[idxs[1][iii]]

            # Use the new predicted matrix as input training data for the GP
            BuildGPstart_time = time.time()
            GPs = []

            res = tmpR

            newX = np.delete(res, myfunc.input_dim, axis=1)
            newY = (np.delete(res, 0, axis=1)).tolist()

            n_in_X = (newX.tolist()).copy()
            n_in_Y = newY.copy()

            for _ in range(myfunc.input_dim - 2):
                newY = (np.delete(newY, 0, axis=1)).tolist()
            newY = ((np.delete(newY, 0, axis=1)).reshape(1, -1)).tolist()

            train_x = torch.tensor(newX, dtype=torch.float32) 
            train_y = torch.tensor(newY, dtype=torch.float32) 
            myGP = ExactGPModel(train_x, train_y) 
            myBO.optimizeHyperparamaters(myGP, train_x, train_y, GPHypers_optim, GPoptimIter)

            BuildGPstop_time = time.time()
            print("Build GPs time:", str(BuildGPstop_time - BuildGPstart_time), " seconds")
        elif method == "Mode":

            R_in = np.append(n_in_X, n_in_Y, axis=1)
            RforIdx = R_in.copy()
            RforIdx[np.isnan(RforIdx)] = -1

            idxs = np.where(RforIdx == -1)

            tmpR = R_in.copy()
            modesR = mode(tmpR, axis=0, nan_policy='omit')[0][0]
            for iii in range(len(idxs[0])):
                tmpR[idxs[0][iii], idxs[1][iii]] = modesR[idxs[1][iii]]

            # Use the new predicted matrix as input training data for the GP
            BuildGPstart_time = time.time()
            GPs = []

            res = tmpR

            newX = np.delete(res, myfunc.input_dim, axis=1)
            newY = (np.delete(res, 0, axis=1)).tolist()
            for _ in range(myfunc.input_dim - 2):
                newY = (np.delete(newY, 0, axis=1)).tolist()
            newY = ((np.delete(newY, 0, axis=1)).reshape(1, -1)).tolist()

            n_in_X = (newX.tolist()).copy()
            n_in_Y = newY.copy()

            train_x = torch.tensor(newX, dtype=torch.float32) 
            train_y = torch.tensor(newY, dtype=torch.float32) 
            myGP = ExactGPModel(train_x, train_y) 
            myBO.optimizeHyperparamaters(myGP, train_x, train_y, GPHypers_optim, GPoptimIter)

            BuildGPstop_time = time.time()
            print("Build GPs time:", str(BuildGPstop_time - BuildGPstart_time), " seconds")
        elif method == "Mean":
            ######################### Random sampling #########################
            testX = [myfunc.randUniformInNBounds() for i in range(10000)]
            testX = torch.tensor(testX) 

            acq, _, _ = myBO.gpucb(testX, myGP, BetaT)

            nextX = testX[np.argmax(acq)]
            nextX = nextX.detach().cpu().numpy()
        elif method == "uGP":
            testX = [myfunc.randUniformInNBounds() for i in range(10000)]
            logprobsX = kde.score_samples(testX)
            probX = torch.tensor(logprobsX) 

            acq, _, _ = myBO.gpucb(probX, myGP, BetaT)

            nextX = testX[np.argmax(acq)]
            print("nextX:", nextX)
        else:
            ######################### Random sampling #########################
            testX = [myfunc.randUniformInNBounds() for i in range(10000)]
            testX = torch.tensor(testX) 

            acq, _, _ = myBO.gpucb(testX, myGP, BetaT)

            nextX = testX[np.argmax(acq)]
            nextX = nextX.detach().cpu().numpy()

        # Query true objective function
        nextY, out_X = myfunc.func_with_missing(myfunc.denormalize(nextX))
        print("Out X:", out_X)

        # Augument data and update the statistical model
        if method == "Drop":
            if np.isnan(out_X).any():
                # We skip the observation if there is a missing value
                print("Drop!!")
                continue
            else:
                # Add to D (DropBO method)
                in_X.append(myfunc.denormalize(nextX))
                n_in_X.append(nextX)
                in_Y.append(nextY)
        elif method == "Suggest" or method == "uGP":
            in_X.append(myfunc.denormalize(nextX))
            n_in_X.append(nextX)
            in_Y.append(nextY)
        else:
            # Add to D
            nout_X = myfunc.normalize(out_X)
            in_X.append(myfunc.denormalize(nout_X))
            n_in_X.append(nout_X)
            in_Y.append(nextY)

        print("Next X:", myfunc.denormalize(nextX), " nextY:", nextY)
        minY = np.min(in_Y)
        maxY = np.max(in_Y)
        if method == "BPMF":
            n_in_Y = [[(((nb - na) * (tmpY - minY) / (maxY - minY) + na))] for tmpY in in_Y]
        elif method == "BOMI" or method == "Mean" or method == "Mode" or method == "KNN":
            n_in_Y = [[(((nb - na) * (tmpY - minY) / (maxY - minY) + na))] for tmpY in in_Y]
            in_Guessed_nY = [
                (((nb - na) * (tmpY - np.min(in_Guessed_Y)) / (np.max(in_Guessed_Y) - np.min(in_Guessed_Y)) + na)) for
                tmpY in in_Guessed_Y]
        else:
            if method == "Suggest" or method == "Drop" or method == "uGP":
                if minY == maxY:
                    minY -= 0.01

            n_in_Y = [(((nb - na) * (tmpY - minY) / (maxY - minY) + na)) for tmpY in in_Y]

            train_x = torch.tensor(n_in_X, dtype=torch.float32)
            train_y = torch.tensor(n_in_Y, dtype=torch.float32)

            myGP = ExactGPModel(train_x, train_y)
        print("Iter ", ite, " Optimum Y: \033[1m", np.max(np.union1d(in_Y, Yori)), "\033[0;0m at: ",
              myfunc.denormalize(n_in_X[np.argmax(in_Y)]))

    BOstop_time = time.time()
    ymax = np.max(np.union1d(in_Y, Yori))
    print("Run:", run, " method: ", method, " y: ", str(ymax), " numCalls: ", myfunc.numCalls, " ite:", ite,
          " time: --- %s seconds ---" % (BOstop_time - BOstart_time))

    ####################### END BO loops #######################

print("Solution: x=", in_X[np.argmax(in_Y)], " f(x)=", np.max(in_Y))



C:\Users\nqduong\Anaconda3\envs\BOMI\lib\site-packages\gpytorch\utils\cholesky.py:24: UserWarning: torch.cholesky is deprecated in favor of torch.linalg.cholesky and will be removed in a future PyTorch release.
L = torch.cholesky(A)
should be replaced with
L = torch.linalg.cholesky(A)
and
U = torch.cholesky(A, upper=True)
should be replaced with
U = torch.linalg.cholesky(A).transpose(-2, -1).conj().
This transform will produce equivalent results for all valid (symmetric positive definite) inputs. (Triggered internally at  ..\aten\src\ATen\native\BatchLinearAlgebra.cpp:1285.)
  L = torch.cholesky(A, upper=upper, out=out)
C:\Users\nqduong\Anaconda3\envs\BOMI\lib\site-packages\ipykernel_launcher.py:122: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


nextX: [0.32226014137268066, 0.15331131219863892, 0.9935303330421448, 0.9648423790931702]  idx: 0
Out X: [-3.5547971725463867, -6.933773756027222, 9.870606660842896, 9.296847581863403]
Next X: [-3.5547971725463867, -6.933773756027222, 9.870606660842896, 9.296847581863403]  nextY: -1.336522902194974
Iter  0  Optimum Y:  577.2784059365059  at:  [-9.769373927810904, nan, -0.6123755622323213, 6.463017433086641]
nextX: [0.4437677562236786, 0.38910698890686035, 0.5124858617782593, 0.9861469864845276]  idx: 0
Out X: [-1.1246448755264282, -2.217860221862793, 0.24971723556518555, nan]
Next X: [-1.1246448755264282, -2.217860221862793, 0.24971723556518555, 9.722939729690552]  nextY: -4.0068851465211255
Iter  1  Optimum Y:  577.2784059365059  at:  [-9.769373927810904, nan, -0.6123755622323213, 6.463017433086641]
nextX: [0.05729268491268158, 0.10001023858785629, 0.006503211334347725, 0.8160138130187988]  idx: 0
Out X: [-8.854146301746368, -7.999795228242874, -9.869935773313046, 6.320276260375977]
N

nextX: [0.3478257656097412, 0.2665744423866272, 0.7783553600311279, 0.9746875762939453]  idx: 0
Out X: [-3.043484687805176, -4.668511152267456, 5.567107200622559, 9.493751525878906]
Next X: [-3.043484687805176, -4.668511152267456, 5.567107200622559, 9.493751525878906]  nextY: 132.91287882464002
Iter  20  Optimum Y:  1149.8470928463462  at:  [-1.6685420274734497, -1.6343390941619873, 5.384366512298584, 9.93251085281372]
nextX: [0.008227942511439323, 0.7119781374931335, 0.48284900188446045, 0.681847095489502]  idx: 0
Out X: [nan, 4.239562749862671, -0.343019962310791, 3.636941909790039]
Next X: [-9.835441149771214, 4.239562749862671, -0.343019962310791, 3.636941909790039]  nextY: 29.328432774512265
Iter  21  Optimum Y:  1149.8470928463462  at:  [-1.6685420274734497, -1.6343390941619873, 5.384366512298584, 9.93251085281372]
nextX: [0.2731361985206604, 0.49158987402915955, 0.7870732545852661, 0.9910438060760498]  idx: 0
Out X: [-4.537276029586792, -0.16820251941680908, 5.741465091705322, n

nextX: [0.5953170657157898, 0.3783543109893799, 0.4678875207901001, 0.9923578500747681]  idx: 0
Out X: [1.906341314315796, -2.4329137802124023, -0.642249584197998, 9.847157001495361]
Next X: [1.906341314315796, -2.4329137802124023, -0.642249584197998, 9.847157001495361]  nextY: 51.15682300589633
Iter  40  Optimum Y:  1149.8470928463462  at:  [-1.6685420274734497, -1.6343390941619873, 5.384366512298584, 9.93251085281372]
nextX: [0.5329283475875854, 0.6011852622032166, 0.5130327939987183, 0.9845896363258362]  idx: 0
Out X: [0.658566951751709, 2.023705244064331, 0.26065587997436523, 9.691792726516724]
Next X: [0.658566951751709, 2.023705244064331, 0.26065587997436523, 9.691792726516724]  nextY: -6.811416252692302
Iter  41  Optimum Y:  1149.8470928463462  at:  [-1.6685420274734497, -1.6343390941619873, 5.384366512298584, 9.93251085281372]
nextX: [0.46236342191696167, 0.7925817370414734, 0.46730807423591614, 0.9779322147369385]  idx: 0
Out X: [-0.7527315616607666, 5.851634740829468, -0.6538

nextX: [0.6251426935195923, 0.7687132954597473, 0.849748969078064, 0.987971305847168]  idx: 0
Out X: [nan, 5.374265909194946, 6.994979381561279, 9.75942611694336]
Next X: [2.5028538703918457, 5.374265909194946, 6.994979381561279, 9.75942611694336]  nextY: -122.85526550880633
Iter  60  Optimum Y:  1149.8470928463462  at:  [-1.6685420274734497, -1.6343390941619873, 5.384366512298584, 9.93251085281372]
nextX: [0.30814826488494873, 0.6694009304046631, 0.6374637484550476, 0.9963017106056213]  idx: 0
Out X: [-3.8370347023010254, 3.3880186080932617, nan, 9.926034212112427]
Next X: [-3.8370347023010254, 3.3880186080932617, 2.749274969100952, 9.926034212112427]  nextY: -18.861706363501206
Iter  61  Optimum Y:  1149.8470928463462  at:  [-1.6685420274734497, -1.6343390941619873, 5.384366512298584, 9.93251085281372]
nextX: [0.11728929728269577, 0.6371434926986694, 0.5561668276786804, 0.9753077030181885]  idx: 0
Out X: [nan, 2.7428698539733887, 1.1233365535736084, 9.50615406036377]
Next X: [-7.6542

...

nextX: [0.2057773768901825, 0.8279619812965393, 0.9317857027053833, 0.3109993636608124]  idx: 0
Out X: [-5.88445246219635, nan, 8.635714054107666, -3.7800127267837524]
Next X: [-5.88445246219635, 6.559239625930786, 8.635714054107666, -3.7800127267837524]  nextY: 6.1337216862639945
Iter  21  Optimum Y:  577.2784059365059  at:  [-9.769373927810904, -0.7663324785952685, -0.6123755622323213, nan]
nextX: [0.4015916585922241, 0.7558916211128235, 0.97260981798172, 0.20061834156513214]  idx: 0
Out X: [-1.9681668281555176, 5.11783242225647, nan, -5.987633168697357]
Next X: [-1.9681668281555176, 5.11783242225647, 9.4521963596344, -5.987633168697357]  nextY: -218.71258717922063
Iter  22  Optimum Y:  577.2784059365059  at:  [-9.769373927810904, -0.7663324785952685, -0.6123755622323213, nan]
nextX: [0.23991413414478302, 0.9455178380012512, 0.9625827074050903, 0.7294109463691711]  idx: 0
Out X: [-5.20171731710434, 8.910356760025024, 9.251654148101807, 4.588218927383423]
Next X: [-5.20171731710434, 8

nextX: [0.18295727670192719, 0.7095655798912048, 0.7143833041191101, 0.7205137014389038]  idx: 0
Out X: [-6.340854465961456, 4.191311597824097, 4.287666082382202, 4.410274028778076]
Next X: [-6.340854465961456, 4.191311597824097, 4.287666082382202, 4.410274028778076]  nextY: 856.7907178865767
Iter  41  Optimum Y:  856.7907178865767  at:  [-6.340854465961456, 4.191311597824097, 4.287666082382202, 4.410274028778076]
nextX: [0.3384677767753601, 0.7766183614730835, 0.7371610403060913, 0.8680300712585449]  idx: 0
Out X: [nan, 5.53236722946167, 4.743220806121826, 7.360601425170898]
Next X: [-3.230644464492798, 5.53236722946167, 4.743220806121826, 7.360601425170898]  nextY: -55.81729446474682
Iter  42  Optimum Y:  856.7907178865767  at:  [-6.340854465961456, 4.191311597824097, 4.287666082382202, 4.410274028778076]
nextX: [0.39631399512290955, 0.9359858632087708, 0.9787875413894653, 0.9925318956375122]  idx: 0
Out X: [-2.073720097541809, 8.719717264175415, 9.575750827789307, nan]
Next X: [-2.0

nextX: [0.40830257534980774, 0.9716421961784363, 0.42069607973098755, 0.4570741653442383]  idx: 0
Out X: [-1.8339484930038452, 9.432843923568726, -1.586078405380249, -0.8585166931152344]
Next X: [-1.8339484930038452, 9.432843923568726, -1.586078405380249, -0.8585166931152344]  nextY: 883.2098108538023
Iter  61  Optimum Y:  883.2098108538023  at:  [-1.8339484930038452, 9.432843923568726, -1.586078405380249, -0.8585166931152344]
nextX: [0.3488050699234009, 0.8742460608482361, 0.4365127384662628, 0.8914991617202759]  idx: 0
Out X: [-3.0238986015319824, nan, -1.2697452306747437, 7.829983234405518]
Next X: [-3.0238986015319824, 7.484921216964722, -1.2697452306747437, 7.829983234405518]  nextY: 20.356878733296348
Iter  62  Optimum Y:  883.2098108538023  at:  [-1.8339484930038452, 9.432843923568726, -1.586078405380249, -0.8585166931152344]
nextX: [0.36752963066101074, 0.9904971122741699, 0.5339933633804321, 0.8690986037254333]  idx: 0
Out X: [nan, 9.809942245483398, 0.6798672676086426, 7.3819

nextX: [0.17737773060798645, 0.8178595304489136, 0.8858073353767395, 0.9393488764762878]  idx: 0
Out X: [-6.452445387840271, 6.3571906089782715, 7.71614670753479, 8.786977529525757]
Next X: [-6.452445387840271, 6.3571906089782715, 7.71614670753479, 8.786977529525757]  nextY: -27.802263709179453
Iter  81  Optimum Y:  883.2098108538023  at:  [-1.8339484930038452, 9.432843923568726, -1.586078405380249, -0.8585166931152344]
nextX: [0.47133395075798035, 0.952524721622467, 0.9311532378196716, 0.9309536814689636]  idx: 0
Out X: [nan, 9.05049443244934, 8.623064756393433, 8.619073629379272]
Next X: [-0.5733209848403931, 9.05049443244934, 8.623064756393433, 8.619073629379272]  nextY: -142.53295102838143
Iter  82  Optimum Y:  883.2098108538023  at:  [-1.8339484930038452, 9.432843923568726, -1.586078405380249, -0.8585166931152344]
nextX: [0.6223775148391724, 0.7821263670921326, 0.7313774824142456, 0.5523453950881958]  idx: 0
Out X: [nan, 5.642527341842651, 4.627549648284912, 1.046907901763916]
Nex

nextX: [0.21327008306980133, 0.9283044934272766, 0.5165098905563354, 0.38119328022003174]  idx: 0
Out X: [-5.734598338603973, nan, 0.330197811126709, -2.3761343955993652]
Next X: [-5.734598338603973, 8.566089868545532, 0.330197811126709, -2.3761343955993652]  nextY: 14.49909821307747
Iter  101  Optimum Y:  883.2098108538023  at:  [-1.8339484930038452, 9.432843923568726, -1.586078405380249, -0.8585166931152344]
nextX: [0.29725220799446106, 0.9972767233848572, 0.4418630301952362, 0.32355421781539917]  idx: 0
Out X: [-4.054955840110779, 9.945534467697144, nan, -3.5289156436920166]
Next X: [-4.054955840110779, 9.945534467697144, -1.1627393960952759, -3.5289156436920166]  nextY: -93.60148370777918
Iter  102  Optimum Y:  883.2098108538023  at:  [-1.8339484930038452, 9.432843923568726, -1.586078405380249, -0.8585166931152344]
nextX: [0.9649580717086792, 0.67926025390625, 0.06423890590667725, 0.2899872064590454]  idx: 0
Out X: [nan, 3.585205078125, -8.715221881866455, -4.200255870819092]
Next 

In [11]:
len(in_Y)

171